<a href="https://colab.research.google.com/github/yegisafari/DenseNet/blob/main/assignmnet4_yeganehsafari_deeplearning_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Deep Learning
#Assignmnet 4
#Yeganeh Safari

1. Read the original DenseNet paper, which can be found here: https://arxiv.org/pdf/
1608.06993.pdf.
2. There are a few custom building blocks that need to be implemented for a DenseNet
model: The composition function, the actual DenseNet block, and the transition block.
These can be defined quite straighforwardly using Keras’ functional API.
3. As DenseNets increase the number of feature maps over each DenseNet block, to make
the model more compact, also a compression factor can be implemented for the transition
block. Here, use a compression factor of θ = 0.5.
4. Note that in the DenseNet paper, a second version is proposed that also includes Bottle-
neck layers. You can experiment with those as well, but they may not be needed for the
relatively small CIFAR-10 Dataset.
5. As actual architecture, use a network that uses a total of 3 DenseNet blocks, with a
growth rate of k = 12. Process the input images as described in the paper.
6. For the classifier part of the network (i.e. the part after the DenseNet blocks) use global
averaging pooling, followed by the dense output layer (as in GoogLeNet).
7. There are two sets of experiments reported in the paper for CIFAR-10. One using data
augmentation (as we have seen in class) but no dropout, and one using dropout but no
data augmentation. Try out both, using a dropout rate of 20%.
8. Evaluate all your models on the test dataset. Which version gives the most accurate
results? Does your DenseNet model beat the other models we have seen in class?

In [ ]:
from tensorflow.keras.layers import BatchNormalization, Activation, Conv2D, concatenate
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, GlobalAveragePooling2D, Dense, concatenate
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D, Dense, Dropout, Flatten, Input, concatenate, BatchNormalization, Activation, GlobalAveragePooling2D
from tensorflow.keras.layers import Input, Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import AveragePooling2D
import tensorflow as tf
from tensorflow.keras.layers import BatchNormalization, Activation
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, Activation, AveragePooling2D, Concatenate, GlobalAveragePooling2D, Dense
import tensorflow.keras.backend as K
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision.models import densenet121

In [ ]:
cifar_10 = tf.keras.datasets.cifar10

In [ ]:
(X_train, Y_train), (X_test, Y_test) = cifar_10.load_data()

In [ ]:
x_train = x_train.astype('float32')
x_train = x_train/255.0

x_test = x_test.astype('float32')
x_test = x_test/255.0

In [ ]:
x_test.shape

(10000, 32, 32, 3)

In [ ]:
y_test.shape

(10000, 1)

In [ ]:
def composition_function(inputs, filter, kernel_size, dropout=0.0):
   x = tf.keras.layers.BatchNormalization()(inputs)
   x = tf.keras.layers.Activation('relu')(x)
   x = tf.keras.layers.Conv2D(filter, kernel_size, padding='same')(x)
   outputs = tf.keras.layers.Dropout(dropout)(x)
   return outputs

In [ ]:
# DenseNet block
def densenet_block(inputs, layers, filters, dropout=0.0, bottleneck=False):
  for _ in range(layers):
    if bottleneck:
      x = composition_function(inputs, 4*filters, (1,1), dropout=dropout)
    else:
      x = inputs
    x = composition_function(x, filters, (3,3), dropout=dropout)
    inputs = tf.keras.layers.Concatenate(axis=-1)([x, inputs])
  return inputs
def densenet_block(inputs, layers, filters, dropout=0.0, bottleneck=False):
  for _ in range(layers):
    if bottleneck:
      x = composition_function(inputs, 4*filters, (1,1), dropout=dropout)
    else:
      x = inputs
    x = composition_function(x, filters, (3,3), dropout=dropout)
    inputs = tf.keras.layers.Concatenate(axis=-1)([x, inputs])
  return inputs

In [ ]:
# Compression fcator alpha = 0.5
def transition_block(inputs, compression=0.5, dropout=0.00):
  filters = tf.keras.backend.int_shape(inputs)[-1]
  filters = int(filters * compression)
  x = composition_function(inputs, filters, (1, 1), dropout=dropout)
  outputs = tf.keras.layers.AveragePooling2D(pool_size=(2,2), strides=(2,2), padding='same')(x)
  return outputs

In [ ]:
#Data augmentation
input_shape = (32, 32, 3)
n_classes = 10
blocks = (6, 12, 24)
growth_rate = 12
compression_rate = 0.5
dropout_rate = 0.0
bottleneck = True

In [ ]:
inputs = tf.keras.Input(shape=input_shape)
x = tf.keras.layers.RandomFlip('horizontal')(inputs)
x = tf.keras.layers.RandomRotation(0.2)(x)

In [ ]:
# Convolution layer
x = tf.keras.layers.Conv2D(2*growth_rate, (5, 5), strides=(2, 2), padding='same')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Activation('relu')(x)
x = tf.keras.layers.MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='same')(x)

In [ ]:
# Dense net
for i, num in enumerate(blocks):
    x = densenet_block(x, num, growth_rate, dropout=dropout_rate, bottleneck = bottleneck)
    if i != len(blocks) - 1:
        x = transition_block(x, compression=compression_rate, dropout=dropout_rate)

x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Activation('relu')(x)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dense(n_classes, activation='softmax')(x)

aug_model = tf.keras.Model(inputs=inputs, outputs=x)

aug_model.summary()

Model: "model_8"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_13 (InputLayer)       [(None, 32, 32, 3)]          0         []                            
                                                                                                  
 random_flip (RandomFlip)    (None, 32, 32, 3)            0         ['input_13[0][0]']            
                                                                                                  
 random_rotation (RandomRot  (None, 32, 32, 3)            0         ['random_flip[0][0]']         
 ation)                                                                                           
                                                                                                  
 conv2d_130 (Conv2D)         (None, 16, 16, 24)           1824      ['random_rotation[0][0]'

In [ ]:
aug_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


aug_hist = aug_model.fit(X_train, Y_train, epochs=2, batch_size=32, validation_split=0.1,
                         callbacks=[tf.keras.callbacks.EarlyStopping(patience=5)])

Epoch 1/2
1407/1407 [==============================] - 160s 83ms/step - loss: 1.9306 - accuracy: 0.2977 - val_loss: 1.7765 - val_accuracy: 0.3684
Epoch 2/2
1407/1407 [==============================] - 113s 80ms/step - loss: 1.6769 - accuracy: 0.3941 - val_loss: 1.6532 - val_accuracy: 0.4138


In [ ]:
# Model Evaluation
loss, accuracy = aug_model.evaluate(X_test, Y_test, verbose=0)
print("Test Loss: ", loss)
print("Test Accuracy: ", accuracy)

Test Loss:  1.6381686925888062
Test Accuracy:  0.41920000314712524


In [ ]:
input_shape = (32, 32, 3)
n_classes = 10
blocks = (6, 12, 24)
growth_rate = 12
compression_rate = 0.5
dropout_rate = 0.2
bottleneck = True

In [ ]:
inputs = tf.keras.Input(shape=input_shape)
x = inputs

In [ ]:
# Convolution layer
x = tf.keras.layers.Conv2D(2 * growth_rate, (7, 7), strides=(2, 2), padding='same')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Activation('relu')(x)
x = tf.keras.layers.MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='same')(x)

In [ ]:
# Dense net
for i, num in enumerate(blocks):
  x = densenet_block(x, num, growth_rate, dropout=dropout_rate, bottleneck=bottleneck)
  if i != len(blocks) - 1:
    x = transition_block(x, compression=compression_rate, dropout=dropout_rate)

x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Activation('relu')(x)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dense(n_classes, activation='softmax')(x)

In [ ]:
model = tf.keras.Model(inputs=inputs, outputs=x)

In [ ]:
model.summary()

Model: "model_9"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_15 (InputLayer)       [(None, 32, 32, 3)]          0         []                            
                                                                                                  
 conv2d_218 (Conv2D)         (None, 16, 16, 24)           3552      ['input_15[0][0]']            
                                                                                                  
 batch_normalization_210 (B  (None, 16, 16, 24)           96        ['conv2d_218[0][0]']          
 atchNormalization)                                                                               
                                                                                                  
 activation_210 (Activation  (None, 16, 16, 24)           0         ['batch_normalization_21

In [ ]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
hist = model.fit(X_train, Y_train, epochs=2, batch_size=32, validation_split=0.1,
                 callbacks=[tf.keras.callbacks.EarlyStopping(patience=5)])

Epoch 1/2
1407/1407 [==============================] - 354s 222ms/step - loss: 1.6182 - accuracy: 0.4004 - val_loss: 1.6476 - val_accuracy: 0.4118
Epoch 2/2
1407/1407 [==============================] - 298s 212ms/step - loss: 1.3042 - accuracy: 0.5306 - val_loss: 1.6030 - val_accuracy: 0.4748


In [ ]:
# Model Evaluation
loss, accuracy = model.evaluate(X_test, Y_test, verbose=0)
print("Test Loss: ", loss)
print("Test Accuracy: ", accuracy)

Test Loss:  1.599199652671814
Test Accuracy:  0.48159998655319214
